# **Project: Prompt Engineering with LangChain and ChatGPT for real-world tasks**

Here we will leverage ChatGPT and LangChain to solve and do a few mini-projects based on some real-world use cases:

- Mini-Project 1: Review Analyst
- Mini-Project 2: Research Paper Analyst



### **Install OpenAI and Langchain dependencies**

In [ ]:
!pip install langchain==0.3.11
!pip install langchain-openai==0.2.12
!pip install langchain-community==0.3.11

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.14
    Uninstalling langchain-0.3.14:
      Successfully uninstalled langchain-0.3.14
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.5 MB/s eta 0:00:00


### **Setup API Tokens**

In [ ]:
from getpass import getpass
OPENAI_KEY = getpass("Please enter your Open AI API key here: ")

Please enter your Open AI API key here: ··········


### **Setup system env variables**

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_KEY


### **Load required dependencies and ChatGPT LLM**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [ ]:
chatgpt = ChatOpenAI(model_name="gpt-4", temperature=0)

## **Mini-Project 1: Review Analyst**

Here we are building an AI system to be able to look at customer reviews and do some complex analysis. for each review get ChatGPT to do the following:

  - Summarize the review. The summary should be at most 3 lines.
  - Highlight both the positives and negatives
  - Display the overall sentiment of the review (positive, negative, neutral)
  - Display a list of 3 - 5 emotions expressed by the customer in the review
  - If the sentiment is positive or neutral write an email and thank them for the review
  - If the sentiment is negative apologize and write an email with an appropriate response

We will try to get the response in a nice structured format using an output parser

### Access Customer Reviews

In [ ]:
reviews = [
    f"""
    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.
    The sound quality is impressively clear with just the right amount of bass.
    It's also waterproof, which tested true during a recent splashing incident.
    Though it's compact, the volume can really fill the space.
    The price was a bargain for such high-quality sound.
    Shipping was also on point, arriving two days early in secure packaging.
    """,
    f"""
    Purchased a new gaming keyboard because of its rave reviews about responsiveness and backlighting.
    It hasn't disappointed. The keys have a satisfying click and the LED colors are vibrant,
    enhancing my gaming experience significantly. Price-wise, it's quite competitive,
    and I feel like I got a good deal. The delivery was swift, and it came well-protected,
    ensuring no damage during transport.
    """,
    f"""
    Ordered a set of wireless earbuds for running, and they've been a letdown.
    The sound constantly cuts out, and the fit is uncomfortable after only a few minutes of use.
    They advertised a 12-hour battery life, but I'm barely getting four hours.
    Considering the cost, I expected better quality and performance.
    They did arrive on time, but the positives end there. I'm already looking into a return.
    """,
    f"""
    The tablet stand I bought was touted as being sturdy and adjustable,
    but it's anything but. It wobbles with the slightest touch,
    and the angles are not holding up as promised. It feels like a breeze could knock it over.
    It was also pricier than others I've seen, which adds to the disappointment.
    It did arrive promptly, but what's the use if the product doesn't meet basic expectations?
    """,
    f"""
    Needed a new kitchen blender, but this model has been a nightmare.
    It's supposed to handle various foods, but it struggles with anything tougher than cooked vegetables.
    It's also incredibly noisy, and the 'easy-clean' feature is a joke; food gets stuck under the blades constantly.
    I thought the brand meant quality, but this product has proven me wrong.
    Plus, it arrived three days late. Definitely not worth the expense.
    """
]

### **Define output parser**

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class ReviewAnalysisResponse(BaseModel):
    summary: str = Field(description="A brief summary of the customer review with maximum 3 lines")
    positives: list = Field(description="A list showing the positives mentioned by the customer in the review if any - max 3 points")
    negatives: list = Field(description="A list showing the negatives mentioned by the customer in the review if any - max 3 points")
    sentiment: str = Field(description="One word showing the sentiment of the review - positive, negative or neutral")
    emotions: list = Field(description="A list of 3 - 5 emotions expressed by the customer in the review")
    email: str = Field(description="Detailed email to the customer based on the sentiment")

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=ReviewAnalysisResponse)

### **Create input prompt for LLM**

In [ ]:
# create the final prompt with formatting instructions from the parser
prompt_txt = """
             Analyze the given customer review below and generate the response based on the instructions
             mentioned below in the format instructions.
             Also remember to write a detailed email response for the email field based on these conditions:
               - email should be addressed to Dear Customer and signed with Service Agent
               - thank them if the review is positive or neutral
               - apologize if the review is negative

             Format Instructions:
             {format_instructions}

             Review:
             {review}
            """
prompt = PromptTemplate(
    template=prompt_txt,
    input_variables=["review"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

### **Create LCEL LLM Chain**

In [ ]:
# create a simple LCEL chain to take the prompt, pass it to the LLM, enforce response format using the parser
chain = (prompt
           |
         chatgpt
           |
         parser)

### **Format the input reviews**

In [ ]:
reviews_formatted = [{'review': review} for review in reviews]
reviews_formatted[0]

{'review': "\n    Just received the Bluetooth speaker I ordered for beach outings, and it's fantastic.\n    The sound quality is impressively clear with just the right amount of bass.\n    It's also waterproof, which tested true during a recent splashing incident.\n    Though it's compact, the volume can really fill the space.\n    The price was a bargain for such high-quality sound.\n    Shipping was also on point, arriving two days early in secure packaging.\n    "}

### **Get responses from LLM**

In [ ]:
responses = chain.map().invoke(reviews_formatted)

### **View LLM Responses**

In [ ]:
responses[0]

ReviewAnalysisResponse(summary='Customer is extremely satisfied with the Bluetooth speaker they ordered for beach outings, praising its sound quality, waterproof feature, compact size, and volume. They also appreciated the price and shipping.', positives=['Impressive sound quality with the right amount of bass', 'Waterproof feature tested and proven', 'Compact size with high volume', 'Good price for high-quality sound', 'Shipping was prompt and packaging secure'], negatives=[], sentiment='positive', emotions=['satisfaction', 'happiness', 'excitement', 'relief', 'appreciation'], email="Dear Customer,\n\nThank you for your positive review. We are thrilled to hear that you are satisfied with your Bluetooth speaker. It's great to know that you found the sound quality impressive and the waterproof feature useful. We are also glad that you appreciated the compact size, high volume, and the price of the product. Your satisfaction is our top priority and we are happy that we could exceed your 

In [ ]:
responses[0].dict()

<ipython-input-13-439f43c77e52>:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  responses[0].dict()


{'summary': 'Customer is extremely satisfied with the Bluetooth speaker they ordered for beach outings, praising its sound quality, waterproof feature, compact size, and volume. They also appreciated the price and shipping.',
 'positives': ['Impressive sound quality with the right amount of bass',
  'Waterproof feature tested and proven',
  'Compact size with high volume',
  'Good price for high-quality sound',
  'Shipping was prompt and packaging secure'],
 'negatives': [],
 'sentiment': 'positive',
 'emotions': ['satisfaction',
  'happiness',
  'excitement',
  'relief',
  'appreciation'],
 'email': "Dear Customer,\n\nThank you for your positive review. We are thrilled to hear that you are satisfied with your Bluetooth speaker. It's great to know that you found the sound quality impressive and the waterproof feature useful. We are also glad that you appreciated the compact size, high volume, and the price of the product. Your satisfaction is our top priority and we are happy that we c

In [ ]:
for response in responses:
  for k,v in response.dict().items():
    print(f"{k}: {v}")
    print('------')
    print('\n')

summary: Customer is extremely satisfied with the Bluetooth speaker they ordered for beach outings, praising its sound quality, waterproof feature, compact size, and volume. They also appreciated the price and shipping.
------


positives: ['Impressive sound quality with the right amount of bass', 'Waterproof feature tested and proven', 'Compact size with high volume', 'Good price for high-quality sound', 'Shipping was prompt and packaging secure']
------


negatives: []
------


sentiment: positive
------


emotions: ['satisfaction', 'happiness', 'excitement', 'relief', 'appreciation']
------


email: Dear Customer,

Thank you for your positive review. We are thrilled to hear that you are satisfied with your Bluetooth speaker. It's great to know that you found the sound quality impressive and the waterproof feature useful. We are also glad that you appreciated the compact size, high volume, and the price of the product. Your satisfaction is our top priority and we are happy that we co

<ipython-input-14-a4819029beda>:2: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  for k,v in response.dict().items():


In [ ]:
import pandas as pd

pd.DataFrame(response.dict() for response in responses)

<ipython-input-16-0eb6c96f9f40>:3: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pd.DataFrame(response.dict() for response in responses)


,summary,positives,negatives,sentiment,emotions,email
0,Customer is extremely satisfied with the Bluet...,[Impressive sound quality with the right amoun...,[],positive,"[satisfaction, happiness, excitement, relief, ...","Dear Customer,\n\nThank you for your positive ..."
1,Customer is satisfied with the gaming keyboard...,"[Keyboard's responsiveness, Vibrant LED colors...",[],positive,"[satisfaction, happiness, contentment]","Dear Customer,\n\nThank you for your positive ..."
2,Customer is disappointed with the wireless ear...,[Product arrived on time],"[Sound constantly cuts out, Uncomfortable fit,...",negative,"[disappointed, frustrated, uncomfortable, diss...","Dear Customer,\n\nWe are truly sorry to hear a..."
3,Customer is disappointed with the tablet stand...,[Product arrived promptly],"[Stand is not sturdy, Angles are not holding u...",negative,"[disappointed, frustrated, unsatisfied]","Dear Customer,\n\nWe are truly sorry to hear a..."
4,The customer is dissatisfied with the kitchen ...,[],[Struggles with anything tougher than cooked v...,negative,"[frustration, disappointment, regret]","Dear Customer,\n\nWe are truly sorry to hear a..."


## **Mini-Project 2: Research Paper Analyst**

Here we will make ChatGPT act as an AI expert and transform the given research paper abstract based on the nature of the audience mentioned below.

- Short summary of maximum 10 lines for a general audience
- Detailed report for a healthcare company. Have bullet points for pros and cons of ethics in Generative AI as mentioned in the paper
- Detailed report for a generative AI company solving healthcare problems. Have bullet points for key points mentioned for Generative AI for text, images and structured data based healthcare

We will try to use `ChatPromptTemplate` so you can have a conversation with ChatGPT for each of the above tasks using conversational prompting

### **Access research paper abstract**

In [ ]:
paper_abstract = f"""
The widespread use of ChatGPT and other emerging technology powered by generative
artificial intelligence (AI) has drawn much attention to potential ethical issues, especially in
high-stakes applications such as healthcare.1–3 However, less clear is how to resolve such
issues beyond following guidelines and regulations that are still under discussion and
development. On the other hand, other types of generative AI have been used to synthesize
images and other types of data for research and practical purposes, which have resolved some
ethical issues and exposed other ethical issues,4,5 but such technology is less often the focus
of ongoing ethical discussions. Here we highlight gaps in current ethical discussions of
generative AI via a systematic scoping review of relevant existing research in healthcare, and
reduce the gaps by proposing an ethics checklist for comprehensive assessment and
transparent documentation of ethical discussions in generative AI development. While the
checklist can be readily integrated into the current peer review and publication system to
enhance generative AI research, it may also be used in broader settings to disclose ethicsrelated considerations in generative AI-powered products (or real-life applications of such
products) to help users establish reasonable trust in their capabilities.

Current ethical discussions on generative AI in healthcare
We conducted a systematic scoping review to analyse current ethical discussions on
generative AI in healthcare. Our search in four major academic research databases for
relevant publications from January 2013 to July 2023 yielded 2859 articles (see Methods for
detailed search strategy and Supplementary Figure S1 for the PRISMA flow diagram), of
which 193 articles were included for analysis based on application data modality (text, image,
or structured data), ethical issues discussed, generative AI involved, and whether generative
AI causes or offers technical solutions for issues raised.

Generative AI for text data-based healthcare
Forty-one of the 193 articles discussed ethical considerations pertaining to generative AI
applications for text data, with 20 articles describing methodological developments or
applications of generative AI and the other 21 articles describing review-type works on this
topic. Although some of these review-type articles used the general term “generative AI”, the
main body and supporting evidence focused on LLMs. Twenty-nine articles had in-depth
discussions on ethical issues, whereas the other 12 articles only briefly touched on some
ethical aspects.
Among the 41 articles, 29 articles focused on discussing ethical issues caused by LLMs (and
specifically by GPT in 16 of the articles), covering a wide range of application scenarios and
considered the application of all 10 ethical principles identified in the review (see Figure 1),
as well as other less discussed concerns such as human-AI interaction, and the rights of
LLMs to be considered as co-authors in scientific papers. One paper only commented briefly
on the need for ethical considerations in LLMs and is summarised in the “Others” category.
Although all ethical principles are equally important, some are discussed more often than
others, e.g., non-maleficence (also referred to in the literature as ‘benevolence’), equity, and
privacy.
Fifteen of the 41 articles aimed to resolve some existing ethical issues (for example,
confidentiality of medical data) by using LLMs and other generative AI (e.g., GAN,
autoencoder or diffusion), such as, to reduce privacy concerns by generating synthetic
medical text, to reduce disparity by providing accessible services and assistance, to detect
health-related misinformation, to generate trusted content, and to improve accountability or
transparency over existing approaches. While most articles focused on either identifying
ethical issues caused by generative AI or proposing generative AI-based solutions, three
articles discussed both to provide a more balanced perspective.

Generative AI for image and structured data-based healthcare
Unlike the diverse application scenarios of generative AI based on text data, for image and
structured data, this use of generative AI focuses on data synthesis and encryption. Hence the
majority of articles discussed the methodological developments of generative AI as giving
rise to a more distinctive and focused set of ethical issues.
5
Notably, of the 98 articles on image data and 58 articles on structured data, more than half
(n=63 for image data and n=33 for structured data) only mentioned ethical considerations as a
brief motivation for methodological developments or as a general discussion point. The rest
included more in-depth discussions or evaluations of ethical issues. Among these 155 articles
(as one article covered multiple modalities), 11 articles were review-type work, where 10
articles reviewed methods that mentioned one or two ethical perspectives, and only one
article24 discussed detailed ethical concerns on generative AI applications.
Resolving privacy issues was the main aim of articles for these two data modalities (n=74 for
image data and n=50 for structured data; see Figure 1), predominantly by generating synthetic
data using GAN. Eight articles on image data and 9 articles on structured data used
generative AI to reduce bias, e.g., by synthesizing data for under-represented subgroups in
existing databases. For both data modalities, we did not see explicit discussions on resolving
autonomy, integrity, or morality issues using generative AI, and for structured data the articles
additionally lacked discussions on trust or transparency.
Only 11 articles for image data selectively discussed some ethical issues that generative AI
can give rise to, without specific discussions regarding autonomy, integrity, or morality. For
structured data, only 4 articles discussed equity, privacy, or data security issues caused by
generative AI. Only two articles on structured data included both the cause and resolving
perspectives by discussing ethical issues that may arise from limitations of methods
proposed, specifically bias induced when synthesizing data in order to resolve privacy issues.
"""

### **Creation of prompt template for paper analysis and transformation**

In [ ]:
SYS_PROMPT = """
Act as a Artificial Intelligence Expert.
Transform the input research paper abstract given below
based on the instruction input by the user.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", "{instruction}"),
    ]
)

### **Create a simple LCEL LLM Chain**


In [ ]:
chain = (prompt
            |
         chatgpt)

### **Generate first summary report**

In [ ]:
from langchain_core.messages import HumanMessage

prompt_txt = f"""
Based on the following research paper abstract,
create the summary report of maximum 10 lines
for a general audience

Abstract:
{paper_abstract}
"""
messages = [HumanMessage(content=prompt_txt)]
user_instruction = {'instruction': messages}

response = chain.invoke(user_instruction)
messages.append(response)

In [ ]:
print(response.content)

The increasing use of AI technologies like ChatGPT in fields like healthcare has raised ethical concerns. However, there is a lack of clarity on how to address these issues beyond existing guidelines and regulations. This study proposes an ethics checklist for assessing and documenting ethical discussions in AI development. The researchers conducted a review of current ethical discussions on AI in healthcare, analyzing 193 articles from 2013 to 2023. They found that while some articles discussed ethical issues caused by AI, others proposed AI-based solutions. The study also found that privacy, equity, and non-maleficence were the most discussed ethical principles. The proposed checklist can be integrated into the peer review and publication system to enhance AI research and can also be used to disclose ethical considerations in AI-powered products.


In [ ]:
response.content

'The increasing use of AI technologies like ChatGPT in fields like healthcare has raised ethical concerns. However, there is a lack of clarity on how to address these issues beyond existing guidelines and regulations. This study proposes an ethics checklist for assessing and documenting ethical discussions in AI development. The researchers conducted a review of current ethical discussions on AI in healthcare, analyzing 193 articles from 2013 to 2023. They found that while some articles discussed ethical issues caused by AI, others proposed AI-based solutions. The study also found that privacy, equity, and non-maleficence were the most discussed ethical principles. The proposed checklist can be integrated into the peer review and publication system to enhance AI research and can also be used to disclose ethical considerations in AI-powered products.'

In [ ]:
messages

[HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

### **Generate second summary report**

Here we will add previous LLM response and the new instructions to the list of messages and send the whole thing to LLM so it has access to the overall historical conversation

In [ ]:
prompt_txt = f"""
Use only the research paper abstract from earlier and create a detailed report for a healthcare company.
In the report, also include bullet points (3 max) for pros and cons of ethics in Generative AI
"""
messages.append(HumanMessage(content=prompt_txt))
user_instruction = {'instruction': messages}
response = chain.invoke(user_instruction)
messages.append(response)

In [ ]:
print(response.content)

Subject: Detailed Report on Ethical Considerations in Generative AI for Healthcare

Dear Team,

This report is based on a recent research paper that focuses on the ethical considerations in the use of generative Artificial Intelligence (AI) in healthcare. The study highlights the increasing use of AI technologies, such as ChatGPT, and the ethical issues that arise from their application, particularly in high-stakes fields like healthcare.

The research paper proposes an ethics checklist as a solution to these ethical dilemmas. This checklist is designed for comprehensive assessment and transparent documentation of ethical discussions during the development of generative AI. It can be integrated into the current peer review and publication system to enhance AI research. Furthermore, it can be used in broader settings to disclose ethical considerations in AI-powered products, thereby helping users establish reasonable trust in their capabilities.

The researchers conducted a systematic s

In [ ]:
messages

[HumanMessage(content='\nBased on the following research paper abstract,\ncreate the summary report of maximum 10 lines\nfor a general audience\n\nAbstract:\n\nThe widespread use of ChatGPT and other emerging technology powered by generative\nartificial intelligence (AI) has drawn much attention to potential ethical issues, especially in\nhigh-stakes applications such as healthcare.1–3 However, less clear is how to resolve such\nissues beyond following guidelines and regulations that are still under discussion and\ndevelopment. On the other hand, other types of generative AI have been used to synthesize\nimages and other types of data for research and practical purposes, which have resolved some\nethical issues and exposed other ethical issues,4,5 but such technology is less often the focus\nof ongoing ethical discussions. Here we highlight gaps in current ethical discussions of\ngenerative AI via a systematic scoping review of relevant existing research in healthcare, and\nreduce the 

### **Generate third summary report**

Here we will add previous LLM response and the new instructions to the list of messages and send the whole thing to LLM so it has access to the overall historical conversation

In [ ]:
prompt_txt = f"""
Use only the research paper abstract from earlier and create a detailed report for a generative AI company solving healthcare problems.
In the report also include sections for key points mentioned around Generative AI for text, images and structured data based healthcare
"""
messages.append(HumanMessage(content=prompt_txt))
user_instruction = {'instruction': messages}
response = chain.invoke(user_instruction)

In [ ]:
print(response.content)

Subject: Detailed Report on Ethical Considerations in Generative AI for Healthcare Applications

Dear Team,

This report is based on a recent research paper that focuses on the ethical considerations in the use of generative Artificial Intelligence (AI) in healthcare. The study highlights the increasing use of AI technologies, such as ChatGPT, and the ethical issues that arise from their application, particularly in high-stakes fields like healthcare.

The research paper proposes an ethics checklist as a solution to these ethical dilemmas. This checklist is designed for comprehensive assessment and transparent documentation of ethical discussions during the development of generative AI. It can be integrated into the current peer review and publication system to enhance AI research. Furthermore, it can be used in broader settings to disclose ethical considerations in AI-powered products, thereby helping users establish reasonable trust in their capabilities.

The researchers conducted a